In [3]:
using LinearAlgebra

A = [2 -7 0;
     5 10 4;
     0 5 2]

eigenvalues, eigenvectors = eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 2.0000000000000004
 4.999999999999987
 7.000000000000014
vectors:
3×3 Matrix{Float64}:
 -0.624695      0.76835    0.703526
 -2.39683e-17  -0.329293  -0.502519
  0.780869     -0.548821  -0.502519

In [5]:
using LinearAlgebra
using DifferentialEquations
using Plots

# ----- problem data -----
A = [
    1.0   0.0   2.0  -1.8   0.0;
    0.0   5.1   0.0  -1.0   3.0;
    1.0   2.0  -3.0   0.0   0.0;
    0.0   1.0  -3.1   4.0   0.0;
   -2.8   0.0   0.0   1.5   1.0
]

# choose your initial condition and time span
x0    = ones(5)             
tspan = (0.0, 5.0)           

# ----- numerical solution (recommended) -----
function f!(dx, x, p, t)
    mul!(dx, A, x)           # dx = A*x
end

prob = ODEProblem(f!, x0, tspan)
sol  = solve(prob, Tsit5(); saveat=0:0.01: tspan[2])   # any stiff/nonstiff solver works

# ----- closed-form via matrix exponential (for verification / exact linear theory) -----
# X(t) = exp(A*t) * x0
exp_solution = t -> exp(A*t) * x0
println(exp_solution(1.0)) 

# compare at final time
println("‖numerical - expm‖ at t = $(tspan[2]): ",
        norm(sol(tspan[2]) .- exp_solution(tspan[2])))

# eigenvalues of A
println("Eigenvalues of A:")
println(eigvals(A))


[-0.7260237756647356, 202.654846514498, 50.131959248566744, 59.894443631869535, 22.161650767992654]
‖numerical - expm‖ at t = 5.0: 2.3653044683620626e8
Eigenvalues of A:
[-2.9236178482981754, -0.15533809641464663, 2.028824312245176, 4.095614975575989, 5.054516656891652]
